In [1]:
# Importing packages
import numpy as np
from sympy import solve, symbols, Eq
from tqdm import tqdm

# Importing Pauli matrices
from pauli_matrices.pauli_matrices import I, X, Y, Z

# Importing functions
from pauli_matrices.pauli_chain import get_Pauli_chain
from hamiltonian.hamiltonian import get_hamiltonian

In [2]:
def get_Bell_term(measurement_choices, measurement_indices):
            
    # Obtaining Pauli chain
    Pauli_chain = get_Pauli_chain(measurement_choices)

    # Obtaining the variable indices
    lower_string = ''

    for j in measurement_indices:
        lower_string += str(j)

    # Finilizing the variable
    var = symbols('c_{' + lower_string + '}')

    # Returning the Bell operator term
    return var * Pauli_chain, var



In [15]:
# Defining qubit transform
qubit_transform = 'JW'

# Obtainig Hamiltonian
hamiltonian = get_hamiltonian(qubit_transform = qubit_transform)

# Extracting parameters
H = hamiltonian.matrix_form
N = hamiltonian.N

# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# Extracting the ground state energy and the respective eigenstate
E_G = eig_vals[0]
psi_G = eig_vecs[:,0]
psi_G_dagger = np.conjugate(np.transpose(psi_G))



In [16]:
E_G

-1.8510456784448643

In [65]:
# Defining measurements of the parties
M1 = M2 = M3 = M4 = [I, X, Y, Z]

# Consrtucting a measurement matrix
M = np.array([
    M1,
    M2,
    M3,
    M4
])

# Constructing a matrix with variables instead of matrices for the Bell inequality
ones = [1]*N
# M_vars = 

# Initializing the Bell operator, the Bell inequality and a string containing the variables
B = 0
var = []

# Creating indices for all measurements
temp_indices = [[i, j, k, l] for i in range(len(M1)) for j in range(len(M2)) for k in range(len(M3)) for l in range(len(M4))]

# Checking for one and two body terms requirement
indices = []
one_and_two_body = False

if one_and_two_body:
    for j in temp_indices:
        if np.sum([j[i] == 0 for i in range(len(j))]) >= 2:
            indices.append(j)
else:
    indices = temp_indices.copy()

# Calculating the Bell operator
for measurement_indices in tqdm(indices):

    # Constructing the measurement choices
    measurement_choices = [M[p, measurement_indices[p]] for p in range(N)]

    # Calculating the Bell terms and the corresponding variable
    B_temp, var_temp = get_Bell_term(measurement_choices, measurement_indices)
    
    # Adding these temporaries to the existsing Bell operator and the var list
    B += B_temp
    var.append(var_temp)
# 

100%|██████████| 256/256 [00:08<00:00, 30.15it/s]


In [66]:
# Obtaining the sytsem of equations (this takes ages)
pauli_matrices = [I, X, Y, Z]

# Initializing the equations list
eqs = []

# Looping over the equations
for measurement_indices in tqdm(indices):

    # Constructing the measurement choices
    measurement_choices = [pauli_matrices[p] for p in measurement_indices]
    Projector = get_Pauli_chain(measurement_choices)

    # calculating system of equations
    eqs.append( 
        Eq( 
            np.trace( np.matmul(B, Projector)).simplify(), 
            np.trace( np.matmul(H, Projector) )
        ) 
    )
   

100%|██████████| 256/256 [13:30<00:00,  3.16s/it]


In [67]:
# Solving the system for equations
ans = solve(eqs, var)

In [70]:
psi_G

array([ 0.        ,  0.        ,  0.        ,  0.11282248,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.99361516,  0.        ,  0.        ,
        0.        ])

In [82]:
# Still need to make the Bell inequality 
# Need to implement the faster algorithm in order to calculate the classical bound
# Still need to add the ground state of the Hamiltonian to the Hamiltonian
# Let's try to implement the algorithm first, as this does require a different notation and only a subpart of the Bell inequality
# I think we can simply model more rows in the measurement matrix and set those to zero
# Let's try it on the very simple XXZZ Hamiltonian first
# I think it is best to rewrite the Bell inequality in a very simple sum. This allows us to compare it to the Bell inequality given by Jura, et al. 2017

array([0, 1, 0, 0])